## **Pomocne moduly**

Generování multiindexových množin

In [67]:
import itertools
import math
import numpy as np
from scipy.special import comb

def setsize(N, w):
    return int(comb(N+w-1, N-1))    

def td_set_recursive(N, w, rows):
    if N == 1:
        subset = w*np.ones([rows, 1])
    else:
        if w == 0:
            subset = np.zeros([rows, N])
        elif w == 1:
            subset = np.eye(N)
        else:
            subset = np.empty([rows, N])
            row_start = 0
            for k in range(0, w+1):
                sub_rows = setsize(N-1, w-k)
                row_end = row_start + sub_rows - 1
                subset[row_start:row_end+1, 0] = k*np.ones(sub_rows)
                subset[row_start:row_end+1, 1:] = td_set_recursive(N-1, w-k, sub_rows)
                row_start = row_end + 1
    return subset

def td_multiindex_set(N, w):
    td_size = int(comb(N+w, N))
    midx_set = np.empty([td_size, N])
    row_start = 0
    for i in range(0, w+1):
        rows = setsize(N, i)
        row_end = rows + row_start - 1
        midx_set[row_start:row_end+1, :] = td_set_recursive(N, i, rows)
        row_start = row_end + 1
    return midx_set.astype(int)

def tp_multiindex_set(N, w):
    orders = np.arange(0, w+1, 1).tolist()
    if N == 1:
        midx_set = np.array(list(map(lambda el:[el], orders)))
    else:
        midx = list(itertools.product(orders, repeat=N))
        midx = [list(elem) for elem in midx]
        midx_sums = [int(math.fsum(midx[i])) for i in range(len(midx))]
        midx_sorted = sorted(range(len(midx_sums)), key=lambda k: midx_sums[k])
        midx_set = np.array([midx[midx_sorted[i]] for i in range(len(midx))])   
    return midx_set.astype(int)

In [69]:
import numpy as np

def admissible_neighbors(index, index_set):
    for_neighbors = forward_neighbors(index)
    for_truefalse = [is_admissible(fn, index_set) for fn in for_neighbors]
    adm_neighbors = np.array(for_neighbors)[for_truefalse].tolist()
    return adm_neighbors

def is_admissible(index, index_set):
    back_neighbors = backward_neighbors(index)
    for ind_b in back_neighbors:
        if ind_b not in index_set:
            return False
    return True

def forward_neighbors(index):
    N = len(index)
    for_neighbors = []
    for i in range(N):
        index_tmp = index[:]
        index_tmp[i] = index_tmp[i] + 1
        for_neighbors.append(index_tmp)
    return for_neighbors

def backward_neighbors(index):
    N = len(index)
    back_neighbors = []
    for i in range(N):
        index_tmp = index[:]
        if index_tmp[i] > 0:
            index_tmp[i] = index_tmp[i] - 1
            back_neighbors.append(index_tmp)
    return back_neighbors


In [71]:
!pip install UQpy
!pip install --upgrade UQpy

## **PCE**

In [73]:

import openturns as ot 

import numpy as np
from UQpy.surrogates import *
from UQpy.distributions import Uniform, JointIndependent
from UQpy.sensitivity import PceSensitivity

class PolynomialChaosExpansionUQPY:
    def __init__(self, pdf, exp_design_in, exp_design_out, degree=4):
        self.pdf = pdf if pdf is not None else JointIndependent([Uniform(0, 1) for _ in range(exp_design_in.shape[1])])
        self.num_inputs = exp_design_in.shape[1]
        self.num_outputs = exp_design_out.shape[1]
        self.num_samples = exp_design_out.shape[0]
        self.exp_design_inputs = exp_design_in
        self.exp_design_outputs = exp_design_out
        
        self.polynomial_basis = TotalDegreeBasis(self.pdf, degree)
        self.regression_method = LeastSquareRegression()
        
        # Vytvoření PCE
        self.pce = PolynomialChaosExpansion(
            polynomial_basis=self.polynomial_basis,
            regression_method=self.regression_method
        )
        self.pce.fit(exp_design_in, exp_design_out)
    
    def predict(self, design_in):
        return self.pce.predict(design_in)
        
    def compute_mean(self):
        mean_value, _ = self.pce.get_moments(higher=False)
        return mean_value
    
    def compute_variance(self):
        _, variance_value = self.pce.get_moments(higher=False)
        return variance_value
    
    def compute_sobol_first(self):
        sensitivity = PceSensitivity(self.pce)
        sensitivity.run()
        return sensitivity.first_order_indices
    
    def compute_sobol_total(self):
        sensitivity = PceSensitivity(self.pce)
        sensitivity.run()
        return sensitivity.total_order_indices

    # Vypočet čísla podmíněnosti matice návrhu, nutné udělat kvuli tomu že jsem zrušil open turns
    def get_condition_number(self):
        design_matrix = self._compute_design_matrix(self.exp_design_inputs)
        if design_matrix is not None:
            return np.linalg.cond(design_matrix)  # Výpočet podmíněnosti
        else:
            raise ValueError("Design matrix could not be computed.")

    def _compute_design_matrix(self, inputs):
        num_samples = inputs.shape[0]
        num_basis = len(self.polynomial_basis.polynomials)
        design_matrix = np.zeros((num_samples, num_basis))
        
        for i in range(num_samples):
            for j in range(num_basis):
                design_matrix[i, j] = self.polynomial_basis.polynomials[j].evaluate(inputs[i])
        
        return design_matrix

    def print_design_matrix(self):
        design_matrix = self._compute_design_matrix(self.exp_design_inputs)
        print("\nMatice návrhu (design_matrix):")
        print(design_matrix)

In [74]:
# Testuje samotné PCE
"""
num_samples = 500
marginals = [Uniform(0, 5), Uniform(0, 2)] 
input_distribution = JointIndependent(marginals)
input_sample = input_distribution.rvs(num_samples)
print("Generovaná vstupní data:")
print(input_sample)

def test_function(X):
    x1, x2 = X[:, 0], X[:, 1]
    return np.expand_dims(2*x1 + 3*x2**2, axis=1)

output_sample = test_function(input_sample)

pce_uqpy = PolynomialChaosExpansionUQPY(input_distribution, input_sample, output_sample)

test_inputs = input_distribution.rvs(10)
predictions = pce_uqpy.predict(test_inputs)

mean_value = pce_uqpy.compute_mean()
variance_value = pce_uqpy.compute_variance()

sobol_first = pce_uqpy.compute_sobol_first()
sobol_total = pce_uqpy.compute_sobol_total()

print("První řád Sobolových indexů:")
print(sobol_first)
print("Celkové Sobolovy indexy:")
print(sobol_total)
print("Střední hodnota modelu:")
print(mean_value)
print("Rozptyl modelu:")
print(variance_value)

true_outputs = test_function(test_inputs)
print("Skutečné hodnoty vs Predikované hodnoty:")
for i in range(len(test_inputs)):
    print(f"Vstupy: {test_inputs[i]}, Skutečná hodnota: {true_outputs[i][0]}, Predikovaná hodnota: {predictions[i][0]}")
    
"""

'\nnum_samples = 500\nmarginals = [Uniform(0, 5), Uniform(0, 2)] \ninput_distribution = JointIndependent(marginals)\ninput_sample = input_distribution.rvs(num_samples)\nprint("Generovaná vstupní data:")\nprint(input_sample)\n\ndef test_function(X):\n    x1, x2 = X[:, 0], X[:, 1]\n    return np.expand_dims(2*x1 + 3*x2**2, axis=1)\n\noutput_sample = test_function(input_sample)\n\npce_uqpy = PolynomialChaosExpansionUQPY(input_distribution, input_sample, output_sample)\n\ntest_inputs = input_distribution.rvs(10)\npredictions = pce_uqpy.predict(test_inputs)\n\nmean_value = pce_uqpy.compute_mean()\nvariance_value = pce_uqpy.compute_variance()\n\nsobol_first = pce_uqpy.compute_sobol_first()\nsobol_total = pce_uqpy.compute_sobol_total()\n\nprint("První řád Sobolových indexů:")\nprint(sobol_first)\nprint("Celkové Sobolovy indexy:")\nprint(sobol_total)\nprint("Střední hodnota modelu:")\nprint(mean_value)\nprint("Rozptyl modelu:")\nprint(variance_value)\n\ntrue_outputs = test_function(test_inputs

## **SAPCE**

In [76]:
class SensitivityAdaptivePCE:
    def __init__(self, pdf, exp_design_in, exp_design_out, max_partial_degree=10):
        if pdf is None:
            chaos_algo_data = ot.FunctionalChaosAlgorithm(exp_design_in, exp_design_out)
            chaos_algo_data.run()
            self.pdf = chaos_algo_data.getDistribution()
        else:    
            self.pdf = pdf
        
        self.exp_design_in = exp_design_in
        self.exp_design_out = exp_design_out
        self.max_partial_degree = max_partial_degree
        
        num_inputs = self.pdf.getDimension()
        td1_set = td_multiindex_set(num_inputs, 1).tolist()
        self.pce = PolynomialChaosExpansion(self.pdf, self.exp_design_in, self.exp_design_out)
        self.pce.set_multi_index_set(td1_set)
        self.pce.construct_basis()
        self.pce.compute_coefficients()
        
        self.active_multi_indices = [self.pce.multi_index_set[0]]
        self.admissible_multi_indices = self.pce.multi_index_set[1:]
        
        admissible_coefficients = self.pce.coefficients[1:].tolist()
        aggregated_admissible_coefficients = np.sum(np.abs(admissible_coefficients), axis=1)
        
        help_index = np.argmax(aggregated_admissible_coefficients)
        max_admissible_multi_index = self.admissible_multi_indices.pop(help_index)
        self.active_multi_indices.append(max_admissible_multi_index)

    # Odtud převzato (zde uděláno): Vypočet čísla podmíněnosti matice návrhu. 




In [77]:
def generate_test_data(num_samples=100, num_inputs=2):

    # Vstupní data: rovnoměrné rozdělení v rozsahu [0, 1]
    input_distribution = ot.ComposedDistribution([ot.Uniform(0, 1)] * num_inputs)
    exp_design_in = np.array(input_distribution.getSample(num_samples))
    exp_design_out = np.sum(exp_design_in, axis=1).reshape(-1, 1)
    return exp_design_in, exp_design_out

def test_sensitivity_adaptive_pce():
    num_samples = 100
    num_inputs = 2
    exp_design_in, exp_design_out = generate_test_data(num_samples, num_inputs)

    pce = PolynomialChaosExpansionUQPY(
        pdf=None,
        exp_design_in=exp_design_in,
        exp_design_out=exp_design_out,
        degree=4
    )

    pce.print_design_matrix()
    print("\nPodmíněnost matice:", pce.get_condition_number())
    print("\nStřední hodnota PCE (automaticky):", pce.compute_mean())
    print("\nRozptyl PCE:", pce.compute_variance())
    print("\nPrvní Sobolovy indexy:", pce.compute_sobol_first())
    print("\nCelkové Sobolovy indexy:", pce.compute_sobol_total())
   
    test_inputs = exp_design_in[:5]  
    predictions = pce.predict(test_inputs)
    true_outputs = np.sum(test_inputs, axis=1).reshape(-1, 1)

    print("\nPorovnání skutečných a predikovaných hodnot:")
    for i in range(len(test_inputs)):
        print(f"Vstupy: {test_inputs[i]}, Skutečná hodnota: {true_outputs[i][0]}, Predikovaná hodnota: {predictions[i][0]}")
    print("\nPROGRAM PROŠEL celý.")

if __name__ == "__main__":
    test_sensitivity_adaptive_pce()


Matice návrhu (design_matrix):
[[ 1.          1.3068747  -0.31337489 ... -0.7980011   0.04800248
  -1.02575337]
 [ 1.          0.85524771 -0.55929041 ...  0.23068368  0.65062664
  -0.83769912]
 [ 1.          1.5641842   0.46002168 ... -1.4256594   0.59232156
   0.67986138]
 ...
 [ 1.          0.43317474 -0.98436831 ...  0.03149826  0.87516474
   0.47269499]
 [ 1.          1.57367015  0.72633844 ... -0.8718964   0.98420985
   0.78194327]
 [ 1.          1.20099058 -0.10768994 ... -0.54656006  0.058877
  -1.24992149]]

Podmíněnost matice: 3.3379735582641756

Střední hodnota PCE (automaticky): 1.0000000000000007

Rozptyl PCE: 0.1666666666666669

První Sobolovy indexy: [[0.5]
 [0.5]]

Celkové Sobolovy indexy: [[0.5]
 [0.5]]

Porovnání skutečných a predikovaných hodnot:
Vstupy: [0.87726223 0.40953646], Skutečná hodnota: 1.2867986913678502, Predikovaná hodnota: 1.2867986913678513
Vstupy: [0.74688875 0.33854677], Skutečná hodnota: 1.0854355129661821, Predikovaná hodnota: 1.0854355129661832
Vs

In [79]:
"""
import openturns as ot
import numpy as np

def test_SensitivityAdaptivePCE():
    dim = 3
    size = 100
    distribution = ot.ComposedDistribution([ot.Uniform(0, 1)] * dim)
    exp_design_in = distribution.getSample(size)
    model = ot.SymbolicFunction(['x1', 'x2', 'x3'], ['x1 + 2*x2 + 3*x3'])
    exp_design_out = model(exp_design_in)
    exp_design_in_np = np.array(exp_design_in)
    exp_design_out_np = np.array(exp_design_out).reshape(-1, 1)
    sapce = SensitivityAdaptivePCE(pdf=None, exp_design_in=exp_design_in_np, exp_design_out=exp_design_out_np, max_partial_degree=10)
    print("PDF:", sapce.pdf)
    print("Active multi-indices:", sapce.get_active_multi_indices())
    print("Admissible multi-indices:", sapce.get_admissible_multi_indices())
    print("PCE coefficients:", sapce.get_coefficients())
    print("PCE multi-index set:", sapce.get_multi_index_set())

test_SensitivityAdaptivePCE()
"""

BeartypeCallHintParamViolation: Method UQpy.surrogates.polynomial_chaos.PolynomialChaosExpansion.PolynomialChaosExpansion.__init__() parameter polynomial_basis="class=JointDistribution name=JointDistribution dimension=3 copula=class=IndependentCopula ...]" violates type hint <class 'UQpy.surrogates.polynomial_chaos.polynomials.baseclass.PolynomialBasis.PolynomialBasis'>, as <class "openturns.model_copula.Distribution"> "class=JointDistribution name=JointDistribution dimension=3 copula=class=IndependentCopula ...]" not instance of <class "UQpy.surrogates.polynomial_chaos.polynomials.baseclass.PolynomialBasis.PolynomialBasis">.